<a href="https://colab.research.google.com/github/Luiz-Malanczyn/BCC_TCC_PSI/blob/main/relatorios/TCC_desempenho_algortimos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proposta de Pesquisa Científica:
##Um Método de Detecção de Mudança baseado em PSI
###Gabriel Luiz Tokarski Soto, Luiz Eduardo Malanczyn de Oliveira
###Orientador: Fabrício Enembreck  
\\
##TCC - SPRINT 1
##Relatório dos testes

###Preparação dos dados

####Importando as bibliotecas utilizadas

In [10]:
import numpy as np
import pandas as pd

####Preparando os arquivos csv disponiveis no repositório da equipe

In [11]:
#Caminho do csv contendo os resultados da execução dos testes realizados no MOA através do algoritmo automatizador.
#O primeiro arquivo representa os dados dos testes de tuning para os parametros dos thresholds.
!wget https://raw.githubusercontent.com/Luiz-Malanczyn/BCC_TCC_PSI/main/csv_tests/results.csv -O results.csv
!wget https://raw.githubusercontent.com/Luiz-Malanczyn/BCC_TCC_PSI/main/csv_tests/results_window.csv -O results_window.csv
!wget https://raw.githubusercontent.com/Luiz-Malanczyn/BCC_TCC_PSI/main/csv_tests/results_others_clean.csv -O results_others_clean.csv
!wget https://raw.githubusercontent.com/Luiz-Malanczyn/BCC_TCC_PSI/main/csv_tests/result_tuned_clean.csv -O result_tuned_clean.csv

path = '/content/results.csv'
path_window = '/content/results_window.csv'
path_others = '/content/results_others_clean.csv'
path_tuned = '/content/result_tuned_clean.csv'

--2023-09-04 23:43:13--  https://raw.githubusercontent.com/Luiz-Malanczyn/BCC_TCC_PSI/main/csv_tests/results.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3240 (3.2K) [text/plain]
Saving to: ‘results.csv’

results.csv         100%[===================>]   3.16K  --.-KB/s    in 0s      

2023-09-04 23:43:14 (47.6 MB/s) - ‘results.csv’ saved [3240/3240]

--2023-09-04 23:43:14--  https://raw.githubusercontent.com/Luiz-Malanczyn/BCC_TCC_PSI/main/csv_tests/results_window.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3241 (3.

&emsp;Os testes foram realizados com 2 conjuntos de streams geradas através do MOA, ambos de mudança abrupta e gradual, o primeiro par de streams, utilizado no tuning do PSI foi criado utilizando uma seed rândomica, enquanto o segundo, utilizado na comparação dos detectores, foi criado com outra.

###Primeiro arquivo, testes de tuning para os parametros de threshold de detecção de aviso e de mudança

In [12]:
df = pd.read_csv(path)

#Tratativa para incluir índices representativos do que cada linha representa, sendo eles os parâmetros utilizados nos testes
contador = 0
parametros = ['(2,5)',
'(2,10)',
'(2, 15)',
'(2, 20)',
'(2, 30)',
'(5, 10)',
'(5, 15)',
'(5, 20)',
'(5,30)',
'(10, 15)',
'(10, 20)',
'(10,30)',
'(15, 20)',
'(15, 30)',
'(20, 30)']

novo_indice = []
tipo = ''
for i in df.index:
    if i % 2 == 0:
      tipo = 'Abrupt'
      if i != 0:
        contador+=1
    else:
      tipo = 'Gradual'
    param = tipo + parametros[contador]
    novo_indice.append(param)

df.index = novo_indice
#Convertendo os dados do dataframe para poder realizar eventuais cálculos e comparações dos valores
df = df.astype(float)
df

,Learning evaluation instances,Evaluation time,Model cost,Learned instances,Detected changes,Detected Warnings,Predicion error,True changes,Delay detection,True changes detected,Input values,Model training instances,Model serialized size
"Abrupt(2,5)",100000.0,4.093750,1.267957e-08,100000.0,6.0,0.0,0.99285,1.0,101.0,1.0,1.0,100000.0,12024.0
"Gradual(2,5)",100000.0,4.312500,1.331516e-08,100000.0,6.0,0.0,0.99285,1.0,101.0,1.0,1.0,100000.0,12024.0
"Abrupt(2,10)",100000.0,4.531250,1.396812e-08,100000.0,5.0,0.0,0.99285,1.0,101.0,1.0,1.0,100000.0,12024.0
"Gradual(2,10)",100000.0,4.484375,1.387816e-08,100000.0,5.0,0.0,0.99285,1.0,101.0,1.0,1.0,100000.0,12024.0
"Abrupt(2, 15)",100000.0,4.171875,1.287295e-08,100000.0,4.0,0.0,0.99285,1.0,101.0,1.0,1.0,100000.0,12024.0
"Gradual(2, 15)",100000.0,4.078125,1.264626e-08,100000.0,4.0,0.0,0.99285,1.0,101.0,1.0,1.0,100000.0,12024.0
"Abrupt(2, 20)",100000.0,4.406250,1.358350e-08,100000.0,4.0,0.0,0.99285,1.0,101.0,1.0,1.0,100000.0,12024.0
"Gradual(2, 20)",100000.0,4.281250,1.325909e-08,100000.0,4.0,0.0,0.99285,1.0,101.0,1.0,1.0,100000.0,12024.0
"Abrupt(2, 30)",100000.0,3.984375,1.231684e-08,100000.0,4.0,1.0,0.99285,1.0,104.0,1.0,1.0,100000.0,12024.0
"Gradual(2, 30)",100000.0,4.093750,1.266964e-08,100000.0,4.0,1.0,0.99285,1.0,104.0,1.0,1.0,100000.0,12024.0


&emsp;Separando os dados úteis para a avaliação em um novo dataframe, os dados escolhidos são:
- A quantidade de mudanças detectadas pelo algoritmo
- A quantidade de avisos detectados pelo algoritmo
- A quantidade de mudanças verdadeiras na stream
- O tempo de atraso para detecção
\
&emsp;Também foi adicionado um campo que representa qual a porcentagem das mudanças detectadas eram de fato mudanças verdadeiras.

In [13]:
#Separando as colunas relevantes no novo dataframe
new_df = df[['Detected changes', 'Detected Warnings', 'True changes', 'Delay detection', 'True changes detected']]

#Adicionando novo campo representando quantos dos acertos detectados eram mudanças de conceito reais.
accuracy = (new_df['True changes']/new_df['Detected changes']) * 100
new_df.loc[:,'True Change Accuracy'] = accuracy
new_df

<ipython-input-13-d595b28f9929>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[:,'True Change Accuracy'] = accuracy


,Detected changes,Detected Warnings,True changes,Delay detection,True changes detected,True Change Accuracy
"Abrupt(2,5)",6.0,0.0,1.0,101.0,1.0,16.666667
"Gradual(2,5)",6.0,0.0,1.0,101.0,1.0,16.666667
"Abrupt(2,10)",5.0,0.0,1.0,101.0,1.0,20.000000
"Gradual(2,10)",5.0,0.0,1.0,101.0,1.0,20.000000
"Abrupt(2, 15)",4.0,0.0,1.0,101.0,1.0,25.000000
"Gradual(2, 15)",4.0,0.0,1.0,101.0,1.0,25.000000
"Abrupt(2, 20)",4.0,0.0,1.0,101.0,1.0,25.000000
"Gradual(2, 20)",4.0,0.0,1.0,101.0,1.0,25.000000
"Abrupt(2, 30)",4.0,1.0,1.0,104.0,1.0,25.000000
"Gradual(2, 30)",4.0,1.0,1.0,104.0,1.0,25.000000


&emsp;Com base nos resultados acima, foi escolhido aplicar um filtro, mantendo assim somente os casos onde a acurácia foi superior a 25%, ordenando o dataframe pelo tempo de delay de maneira crescente e ignorando os casos onde não se detectou nenhum aviso. Exibindo assim os 10 melhores resultados.

In [14]:
min_acc = (new_df['True Change Accuracy'] >= 25)
min_warning = (new_df['Detected Warnings'] >= 1)
min_acc_df = new_df[min_acc & min_warning]
min_acc_df = min_acc_df.sort_values(by='Delay detection')
min_acc_df

,Detected changes,Detected Warnings,True changes,Delay detection,True changes detected,True Change Accuracy
"Abrupt(2, 30)",4.0,1.0,1.0,104.0,1.0,25.0
"Gradual(2, 30)",4.0,1.0,1.0,104.0,1.0,25.0
"Abrupt(5,30)",4.0,1.0,1.0,104.0,1.0,25.0
"Gradual(5,30)",4.0,1.0,1.0,104.0,1.0,25.0
"Abrupt(10,30)",4.0,1.0,1.0,104.0,1.0,25.0
"Gradual(10,30)",4.0,1.0,1.0,104.0,1.0,25.0
"Abrupt(15, 30)",4.0,1.0,1.0,104.0,1.0,25.0
"Gradual(15, 30)",4.0,1.0,1.0,104.0,1.0,25.0
"Abrupt(20, 30)",4.0,1.0,1.0,104.0,1.0,25.0
"Gradual(20, 30)",4.0,1.0,1.0,104.0,1.0,25.0


###Segundo arquivo, testes de tuning para o arquivo com os testes de tamanho da janela

&emsp;Com os resultados obtidos no teste anterior, foram escolhidos os 10 melhores resultados para servirem de guia para comparar os resultados de janelas de tamanho diferentes

In [15]:
df_window = pd.read_csv(path_window)
contador = 0

#Assim como no teste anterior, aplica-se uma tratativa para nomear os índices conforme os parametros testados
parametros = [
'(100, 2, 30)',
'(100, 5,30)',
'(100, 10,30)',
'(100, 15, 30)',
'(100, 20, 30)',
'(500, 2, 30)',
'(500, 5,30)',
'(500, 10,30)',
'(500, 15, 30)',
'(500, 20, 30)',
'(1000, 2, 30)',
'(1000, 5,30)',
'(1000, 10,30)',
'(1000, 15, 30)',
'(1000, 20, 30)'
]

novo_indice = []
tipo = ''
for i in df_window.index:
    if i % 2 == 0:
      tipo = 'Abrupt'
      if i != 0:
        contador+=1
    else:
      tipo = 'Gradual'
    param = tipo + parametros[contador]
    novo_indice.append(param)

df_window.index = novo_indice
df_window = df_window.astype(float)
df_window

,Learning evaluation instances,Evaluation time,Model cost,Learned instances,Detected changes,Detected Warnings,Predicion error,True changes,Delay detection,True changes detected,Input values,Model training instances,Model serialized size
"Abrupt(100, 2, 30)",100000.0,4.625000,1.424700e-08,100000.0,4.0,1.0,0.99285,1.0,104.0,1.0,1.0,100000.0,12024.0
"Gradual(100, 2, 30)",100000.0,4.078125,1.261195e-08,100000.0,4.0,1.0,0.99285,1.0,104.0,1.0,1.0,100000.0,12024.0
"Abrupt(100, 5,30)",100000.0,4.218750,1.305789e-08,100000.0,4.0,1.0,0.99285,1.0,104.0,1.0,1.0,100000.0,12024.0
"Gradual(100, 5,30)",100000.0,4.281250,1.323461e-08,100000.0,4.0,1.0,0.99285,1.0,104.0,1.0,1.0,100000.0,12024.0
"Abrupt(100, 10,30)",100000.0,4.437500,1.369002e-08,100000.0,4.0,1.0,0.99285,1.0,104.0,1.0,1.0,100000.0,12024.0
"Gradual(100, 10,30)",100000.0,4.515625,1.387186e-08,100000.0,4.0,1.0,0.99285,1.0,104.0,1.0,1.0,100000.0,12024.0
"Abrupt(100, 15, 30)",100000.0,4.234375,1.309039e-08,100000.0,4.0,1.0,0.99285,1.0,104.0,1.0,1.0,100000.0,12024.0
"Gradual(100, 15, 30)",100000.0,4.687500,1.444488e-08,100000.0,4.0,1.0,0.99285,1.0,104.0,1.0,1.0,100000.0,12024.0
"Abrupt(100, 20, 30)",100000.0,4.562500,1.408810e-08,100000.0,4.0,1.0,0.99285,1.0,104.0,1.0,1.0,100000.0,12024.0
"Gradual(100, 20, 30)",100000.0,4.343750,1.340807e-08,100000.0,4.0,1.0,0.99285,1.0,104.0,1.0,1.0,100000.0,12024.0


&emsp;Separando do novo dataframe somente os dados relevantes como mencionado anteriormente na tratativa do primeiro teste, obtem-se os dados do teste do tamanho da janela

In [16]:
new_df_window = df_window[['Detected changes', 'Detected Warnings', 'True changes', 'Delay detection', 'True changes detected']]
accuracy = (new_df_window['True changes']/new_df_window['Detected changes']) * 100
new_df_window.loc[:,'True Change Accuracy'] = accuracy
new_df_window

<ipython-input-16-441461128114>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_window.loc[:,'True Change Accuracy'] = accuracy


,Detected changes,Detected Warnings,True changes,Delay detection,True changes detected,True Change Accuracy
"Abrupt(100, 2, 30)",4.0,1.0,1.0,104.0,1.0,25.0
"Gradual(100, 2, 30)",4.0,1.0,1.0,104.0,1.0,25.0
"Abrupt(100, 5,30)",4.0,1.0,1.0,104.0,1.0,25.0
"Gradual(100, 5,30)",4.0,1.0,1.0,104.0,1.0,25.0
"Abrupt(100, 10,30)",4.0,1.0,1.0,104.0,1.0,25.0
"Gradual(100, 10,30)",4.0,1.0,1.0,104.0,1.0,25.0
"Abrupt(100, 15, 30)",4.0,1.0,1.0,104.0,1.0,25.0
"Gradual(100, 15, 30)",4.0,1.0,1.0,104.0,1.0,25.0
"Abrupt(100, 20, 30)",4.0,1.0,1.0,104.0,1.0,25.0
"Gradual(100, 20, 30)",4.0,1.0,1.0,104.0,1.0,25.0


&emsp;Observa-se que somente nos testes onde o tamanho da janela foi igual a 500, a quantidade de avisos detectados apresentou mudanças, enquanto para janelas 100 e 1000 todos os resultados se mantiveram iguais, sendo 1 para tamanho 100 e 0 para tamanho 1000.\
&emsp;Sendo assim, obtivemos a média de avisos detectados nos casos de janela igual a 500, sendo o valor dessa média 6.2 e selecionamos os parametros onde o número de avisos mais se aproximava dessa média. Estes parâmetros sendo os limites de aviso igual a 10, e de mudança igual a 30.

###Testes comparativos com os outros detectores

&emsp;onforme comentado no teste anterior, foram escolhidos 3 testes realizados com o PSI para comparar com os principais detectores de mudança de conceito, sendo esses DDM, EDDM, ADWIN, CUSUM e Page-Hinkley.

In [25]:
#Aqui fazemos a tratativa dos índices para facilitar a leitura e identificação dos dados nos dataframes

#Principais algoritmos com parametros default
df_others = pd.read_csv(path_others)

contador = 0
parametros = [
'(DDM)',
'(EDDM)',
'(ADWIN)',
'(CUSUM)',
'(PAGE-HINKLEY)'
]

novo_indice = []
tipo = ''
for i in df_others.index:
    if i % 2 == 0:
      tipo = 'Abrupt'
      if i != 0:
        contador+=1
    else:
      tipo = 'Gradual'
    param = tipo + parametros[contador]
    novo_indice.append(param)

df_others.index = novo_indice
df_others = df_others.astype(float)

#PSI com os parametros selecionados
df_tuned = pd.read_csv(path_tuned)

contador = 0
parametros = [
'(100, 10,30)',
'(500, 10,30)',
'(1000, 10,30)',
]

novo_indice = []
tipo = ''
for i in df_tuned.index:
    if i % 2 == 0:
      tipo = 'Abrupt'
      if i != 0:
        contador+=1
    else:
      tipo = 'Gradual'
    param = tipo + parametros[contador]
    novo_indice.append(param)

df_tuned.index = novo_indice
df_tuned = df_tuned.astype(float)

#junção dos dataframes
df_detectors = pd.concat([df_others, df_tuned])
df_detectors

,Learning evaluation instances,Evaluation time,Model cost,Learned instances,Detected changes,Detected Warnings,Predicion error,True changes,Delay detection,True changes detected,Input values,Model training instances,Model serialized size
Abrupt(DDM),100000.0,4.812500,6.962024e-09,100000.0,3.0,1.0,0.004850,1.0,235.0,1.0,1.0,100000.0,5592.0
Gradual(DDM),100000.0,4.234375,6.125677e-09,100000.0,3.0,1.0,0.004850,1.0,235.0,1.0,1.0,100000.0,5592.0
Abrupt(EDDM),100000.0,3.875000,4.571242e-09,100000.0,13.0,0.0,0.034545,1.0,76.0,1.0,1.0,100000.0,4560.0
Gradual(EDDM),100000.0,3.937500,4.644971e-09,100000.0,13.0,0.0,0.034545,1.0,76.0,1.0,1.0,100000.0,4560.0
Abrupt(ADWIN),100000.0,4.859375,8.990109e-09,100000.0,12.0,0.0,0.005339,1.0,365.0,1.0,1.0,100000.0,7456.0
Gradual(ADWIN),100000.0,4.906250,9.075771e-09,100000.0,12.0,0.0,0.005339,1.0,365.0,1.0,1.0,100000.0,7456.0
Abrupt(CUSUM),100000.0,4.484375,6.515183e-09,100000.0,2.0,0.0,0.004875,1.0,340.0,1.0,1.0,100000.0,5616.0
Gradual(CUSUM),100000.0,4.765625,6.923801e-09,100000.0,2.0,0.0,0.004875,1.0,340.0,1.0,1.0,100000.0,5616.0
Abrupt(PAGE-HINKLEY),100000.0,4.609375,7.116533e-09,100000.0,2.0,0.0,0.004854,1.0,346.0,1.0,1.0,100000.0,5968.0
Gradual(PAGE-HINKLEY),100000.0,4.625000,7.140657e-09,100000.0,2.0,0.0,0.004854,1.0,346.0,1.0,1.0,100000.0,5968.0


&emsp;A seguir, obtém-se o novo dataframe de comparação dos detectores, filtrando os campos desejados conforme feito nos testes anteriores.

In [26]:
new_df_detectors = df_detectors[['Detected changes', 'Detected Warnings', 'True changes', 'Delay detection', 'True changes detected']]
accuracy = (new_df_detectors['True changes']/new_df_detectors['Detected changes']) * 100
new_df_detectors.loc[:,'True Change Accuracy'] = accuracy
new_df_detectors

<ipython-input-26-5223d384bb45>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df_detectors.loc[:,'True Change Accuracy'] = accuracy


,Detected changes,Detected Warnings,True changes,Delay detection,True changes detected,True Change Accuracy
Abrupt(DDM),3.0,1.0,1.0,235.0,1.0,33.333333
Gradual(DDM),3.0,1.0,1.0,235.0,1.0,33.333333
Abrupt(EDDM),13.0,0.0,1.0,76.0,1.0,7.692308
Gradual(EDDM),13.0,0.0,1.0,76.0,1.0,7.692308
Abrupt(ADWIN),12.0,0.0,1.0,365.0,1.0,8.333333
Gradual(ADWIN),12.0,0.0,1.0,365.0,1.0,8.333333
Abrupt(CUSUM),2.0,0.0,1.0,340.0,1.0,50.000000
Gradual(CUSUM),2.0,0.0,1.0,340.0,1.0,50.000000
Abrupt(PAGE-HINKLEY),2.0,0.0,1.0,346.0,1.0,50.000000
Gradual(PAGE-HINKLEY),2.0,0.0,1.0,346.0,1.0,50.000000


###Conclusão

&emsp;Conforme o dataframe acima, é possível perceber que aumentando o valor da janela do PSI para além do que foi inicialmente estipulado como padrão, janela de tamanho 100, o método é capaz de detectar com precisão a quantidade de mudanças ocorridas, porém perde a detecção dos avisos, além de apresentar um tempo de atraso maior para a detecção. O caso de tamanho 1000, apresentou um atraso menor do que o do teste de janela com tamanho 500, o menor tendo a mesma taxa de acurácia que o Page-Hinkley, enquanto o teste com a maior janela obteve maior acurácia. \
&emsp;Podemos notar que o PSI obteve melhores resultados que os outros detectores quando leva-se em conta apenas a acurácia, porém com exceção do teste do PSI com janela 100, os outros dois casos apresentaram atraso na detecção significativamente maiores.\
&emsp;Como próximos passos, é preciso validar os testes realizados para replicar os resultados, podendo assim comprovar a eficácia do PSI e seguir para a próxima sprint.